# RB1 and EGFR Fisher test


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats

import cptac
import cptac.utils as u

In [4]:
#cptac.download(dataset='ccrcc', version='0.0')
brain = cptac.Gbm()

In [67]:
gene = 'EGFR'

EGFR_cnv = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'CNV', omics_genes = gene)

EGFR_cnv['EGFR_binary_mutations'] = np.where(
                EGFR_cnv[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'EGFR_Wildtype', 'EGFR_Mutated')
EGFR_tumors = EGFR_cnv[EGFR_cnv.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+'_CNV', gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
EGFR_mut_status = EGFR_tumors.drop(columns_to_drop, axis = 1)

In [68]:
gene = 'RB1' 

RB1_transrcpt_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'transcriptomics', omics_genes = gene)
RB1_reduced_t_mutations = brain.reduce_multiindex(RB1_transrcpt_mutations, levels_to_drop=1) #single col labels

RB1_reduced_t_mutations['RB1_binary_mutations'] = np.where(
                RB1_reduced_t_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'RB1_Wildtype', 'RB1_Mutated')
RB1_tumors = RB1_reduced_t_mutations[RB1_reduced_t_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+'_transcriptomics', gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
RB1_mut_status = RB1_tumors.drop(columns_to_drop, axis = 1)

In [69]:
RB1_EGFR = RB1_mut_status.join(EGFR_mut_status)

rb1_wt = RB1_EGFR.loc[RB1_EGFR['RB1_binary_mutations'] == 'RB1_Wildtype']
rb1_wt_egfr_wt = rb1_wt.loc[rb1_wt['EGFR_binary_mutations'] == 'EGFR_Wildtype']
both_wt_count = len(rb1_wt_egfr_wt)
print("RB1 wt/EGFR wt: ", both_wt_count)

rb1_wt_egfr_mut = rb1_wt.loc[rb1_wt['EGFR_binary_mutations'] == 'EGFR_Mutated']
rb1_wt_egfr_mut_count = len(rb1_wt_egfr_mut)
print("RB1 wt/EGFR mut: ", rb1_wt_egfr_mut_count)

RB1 wt/EGFR wt:  75
RB1 wt/EGFR mut:  15


In [70]:
rb1_mut = RB1_EGFR.loc[RB1_EGFR['RB1_binary_mutations'] == 'RB1_Mutated']
rb1_mut_egfr_wt = rb1_mut.loc[rb1_mut['EGFR_binary_mutations'] == 'EGFR_Wildtype']
rb1_mut_egfr_wt_count = len(rb1_mut_egfr_wt)
print("RB1 mut/EGFR wt: ", rb1_mut_egfr_wt_count)

rb1_mut_egfr_mut = rb1_mut.loc[rb1_mut['EGFR_binary_mutations'] == 'EGFR_Mutated']
both_mut_count = len(rb1_mut_egfr_mut)
print("RB1 mut/EGFR mut: ", both_mut_count)

RB1 mut/EGFR wt:  9
RB1 mut/EGFR mut:  1


In [73]:
# one sided -less
oddsratio, pvalue = scipy.stats.fisher_exact([[both_wt_count,rb1_wt_egfr_mut_count],[rb1_mut_egfr_wt_count, both_mut_count]], alternative='less')
pvalue

0.4997723737779574